# LLMのRAGを使ったQABOT

1. **データ取得**:
    - ICML 2024の論文PDFを取得し、./icml_2024_papersディレクトリに格納する
2. **データの埋め込みベクトル化**:
    - 取得したPDFからテキストを抽出し、ベクトル化する。
3. **検索インデックスの構築**:
    - ベクトル化されたデータを元に検索インデックスを構築する。    <---- now
4. **LLMのRAGモデルの設定**:
    - 質問に対して関連する文書を検索し、その文書を基に回答を生成するRAGモデルを設定する。
5. **QABOTの構築**:
    - ユーザーインターフェースを作成し、ユーザーからの質問に対してRAGモデルを使って回答するQABOTを構築する。

In [1]:
!pip3 install faiss-gpu

In [2]:
import faiss
import pandas as pd
import numpy as np
import os

In [3]:
# 保存ディレクトリ
save_dir = "./icml_2024_embeddings"

# ディレクトリ内のファイル一覧を取得
file_list = os.listdir(save_dir)
file_list

['A Bias-Variance-Covariance Decomposition of Kernel Scores for Generative Models_embeddings.csv',
 'A Closer Look at the Limitations of Instruction Tuning_embeddings.csv',
 'A Human-Inspired Reading Agent with Gist Memory of Very Long Contexts_embeddings.csv',
 'A Language Model’s Guide Through Latent Space_embeddings.csv',
 'A Multimodal Automated Interpretability Agent_embeddings.csv',
 'A Sober Look at LLMs for Material Discovery: Are They Actually Good for Bayesian Optimization Over Molecules?_embeddings.csv',
 'A Tale of Tails: Model Collapse as a Change of Scaling Laws_embeddings.csv',
 'Active Preference Learning for Large Language Models_embeddings.csv',
 'Adaptive Text Watermark for Large Language Models_embeddings.csv',
 'Zero-Shot ECG Classification with Multimodal Learning and Test-time Clinical Knowledge Enhancement_embeddings.csv']

In [4]:
# Faissインデックスの初期化
dimension = None
index = None
file_mapping = []

In [5]:
# CSVファイルを探して一つ選択
file_name = next((f for f in file_list if f.endswith('.csv')), None)

In [6]:
for file_name in file_list:
    file_path = os.path.join(save_dir, file_name)

    # データフレームを読み込む
    df = pd.read_csv(file_path)

    # 埋め込みベクトルを抽出してNumPy配列に変換
    embeddings = df.to_numpy().astype('float32')

    # NumPy 配列を C-contiguous に変換
    embeddings = np.ascontiguousarray(embeddings)

    if dimension is None:
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2距離を使用するインデックスを作成

    # 埋め込みベクトルをインデックスに追加
    index.add(embeddings)

    # ファイル名と埋め込みベクトルのマッピングを保存
    for _ in range(embeddings.shape[0]):
        file_mapping.append(file_name)

In [7]:
# インデックスの保存
faiss.write_index(index, "faiss_index.index")

# マッピング情報の保存
mapping_df = pd.DataFrame(file_mapping, columns=['file_name'])
mapping_df.to_csv("file_mapping.csv", index=False)

print(f"Faiss index has been created and saved using all CSV files in {save_dir}.")

Faiss index has been created and saved using all CSV files in ./icml_2024_embeddings.
